#**Transformers Approach**
*State-of-the-art Machine Learning for PyTorch, TensorFlow and JAX.*
*Transformers provides APIs to easily download and train state-of-the-art pretrained models. Using pretrained models can reduce your compute costs, carbon footprint, and save you time from training a model from scratch. The models can be used across different modalities.*

In [ ]:
! pip install transformers[sentencepiece] -Uqq

In [ ]:
! pip install pandas
! pip install numpy
! pip install matplotlib
! pip install scikit-learn

In [ ]:
import numpy as np
from transformers import AutoTokenizer
from transformers import AutoModel
from transformers import AutoModelForSequenceClassification
from transformers import Trainer
from transformers import TrainingArguments
from transformers import DataCollatorWithPadding

In [ ]:
import pandas as pd

*Add the file name of the dataset*

In [ ]:
file_name = '<file_name>.csv'

In [ ]:
dataset = pd.read_csv(file_name)

In [ ]:
import matplotlib.pyplot as plt
dataset['labels'].hist()

In [ ]:
dataset.count()

feedback    3737
labels      3737
dtype: int64

In [ ]:
from sklearn.model_selection import StratifiedShuffleSplit

In [ ]:
def preprocess(dataset,data_column):
  """Performs data preprocessing on the given dataset."""
  dataset[f"{data_column}_processed"] = dataset[data_column].apply(lambda x: " ".join(x.lower() for x in x.split()))
  dataset[f"{data_column}_processed"]  = dataset[f"{data_column}_processed"].str.replace('[^\w\s]','')
  return dataset

In [ ]:
dataset = preprocess(dataset, 'feedback')

*To remove null rows*

In [ ]:
dataset = dataset.dropna(how='any')

*At least 2 words per review*

In [ ]:
dataset = dataset.loc[(dataset['feedback_processed'].str.count(' ') > 1)]

In [ ]:
dataset = dataset.reset_index(drop=True)

In [ ]:
dataset.count()

feedback              3273
labels                3273
feedback_processed    3273
dtype: int64

In [ ]:
dataset['labels'].hist()

In [ ]:
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
dataset['labels'] = label_encoder.fit_transform(dataset['labels'])

In [ ]:
dataset['labels'].count()

3273

In [ ]:
import pyarrow as pa
s = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
for train_index, test_index in s.split(dataset, dataset['labels']):
  train_set = dataset.loc[train_index]
  test_set = dataset.loc[test_index]

In [ ]:
train_set.hist()
test_set.hist()

In [ ]:
!pip install folium==0.2.1 -Uqq

     |████████████████████████████████| 69 kB 7.9 MB/s 


In [ ]:
! pip install datasets -Uqq

     |████████████████████████████████| 325 kB 36.6 MB/s 
     |████████████████████████████████| 1.1 MB 53.1 MB/s 
     |████████████████████████████████| 136 kB 73.8 MB/s 
     |████████████████████████████████| 212 kB 67.4 MB/s 
     |████████████████████████████████| 127 kB 74.5 MB/s 
     |████████████████████████████████| 94 kB 4.6 MB/s 
     |████████████████████████████████| 271 kB 68.0 MB/s 
     |████████████████████████████████| 144 kB 69.3 MB/s 


In [ ]:
import datasets
d = datasets.DatasetDict({"train":datasets.Dataset(pa.Table.from_pandas(train_set)),"test":datasets.Dataset(pa.Table.from_pandas(test_set))})


In [ ]:
d.reset_format()

In [ ]:
checkpoint = "distilbert-base-uncased"

tokenizer = AutoTokenizer.from_pretrained(checkpoint)

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/483 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

In [ ]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [ ]:
def tokenize_function(examples):
    return tokenizer(examples["feedback_processed"], truncation=True)


tokenized_datasets = d.map(tokenize_function, batched=True)

  0%|          | 0/3 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [ ]:
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['feedback', 'labels', 'feedback_processed', '__index_level_0__', 'input_ids', 'attention_mask'],
        num_rows: 2618
    })
    test: Dataset({
        features: ['feedback', 'labels', 'feedback_processed', '__index_level_0__', 'input_ids', 'attention_mask'],
        num_rows: 655
    })
})

In [ ]:
tokenized_datasets = tokenized_datasets.remove_columns(["feedback", "__index_level_0__", "feedback_processed"])
tokenized_datasets.set_format("torch")

In [ ]:
tokenized_datasets['train'][0]

{'attention_mask': tensor([1, 1, 1, 1, 1, 1, 1, 1]),
 'input_ids': tensor([  101, 14044,  2007,  2832,  1998,  3274,  2147,   102]),
 'labels': tensor(2)}

In [ ]:
full_train_dataset = tokenized_datasets["train"]
full_eval_dataset = tokenized_datasets["test"]


In [ ]:
from sklearn.metrics import accuracy_score, f1_score, recall_score
from datasets import load_metric

In [ ]:
metric = load_metric("accuracy")
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=3)

# Hyper parameters for training
*if not provided default values are used*
[more info ](https://huggingface.co/docs/transformers/main_classes/trainer#transformers.TrainingArguments)

1. learning_rate
2. num_train_epochs
3. per_device_train_batch_size
4. seed
5. optim="adamw_torch" (default optimizer "adamw_hf")





In [ ]:
training_args = TrainingArguments("test_trainer",
                                  learning_rate=5.691013656357132e-06,
                                  num_train_epochs=5,
                                  per_device_train_batch_size=8,
                                  seed=36,
                                  optim="adamw_torch"
                                  )

*The Trainer class provides an API for feature-complete training in PyTorch for most standard use cases [more info](https://huggingface.co/docs/transformers/main_classes/trainer)*

In [ ]:
trainer = Trainer( 
                  model=model,
                  args=training_args, 
                  compute_metrics=compute_metrics,
                  train_dataset=full_train_dataset,
                  eval_dataset=full_eval_dataset,
                  tokenizer=tokenizer,
                  data_collator=data_collator
                  )
trainer.train();

*Runs the evaluation against the test set*

In [ ]:
trainer.evaluate()

*To save the trained model weights to current environment*

In [ ]:
trainer.save_model()

*Load the trained model from the current environment*

In [ ]:
from transformers import pipeline 
classifier = pipeline('sentiment-analysis', model='test_trainer')

In [ ]:
import string
def preprocess_incoming(str_data):
  return str_data.translate(str.maketrans('', '', string.punctuation)).lower()

In [ ]:
def predict(string):
  data = preprocess_incoming(string)
  prediction = classifier(string)
  if prediction[0]['label'] == "LABEL_1":
    # return "NEUTRAL"
    return 1
  elif prediction[0]['label'] == "LABEL_0":
    # return "NEGATIVE"
    return 0
  elif prediction[0]['label'] == "LABEL_2":
    # return "POSITIVE"
    return 2
  

*Loading a validation dataset*

In [ ]:
VALIDATION_SET = pd.read_csv("First dataset.csv")

In [ ]:
from sklearn.preprocessing import LabelEncoder
label_encoder=LabelEncoder()
VALIDATION_SET['labels'] = label_encoder.fit_transform(VALIDATION_SET['labels'])
y_valid = VALIDATION_SET['labels']
y_valid = y_valid.tolist()

In [ ]:
y_pred = []
for i in VALIDATION_SET['feedback']:
  y_pred.append(predict(i))

*Validation accuracy*

In [ ]:
accuracy_score(y_valid, y_pred)

0.8606965174129353

In [ ]:
classifier(["good experience", "bad experience", "normal"])